In [1]:
from flow_dataframe import flow_from_dataframe

import numpy as np
import pandas as pd
import os
import sys
from keras.models import load_model
import tensorflow as tf

Using TensorFlow backend.


In [4]:
with tf.device('/cpu:0'):
    model = load_model('results/mae_11/bone_age_weights.best.hdf5')

In [2]:
class_str_col = 'boneage'
gender_str_col = 'male'

IMG_SIZE = (299, 299)
BATCH_SIZE = 32


In [7]:
base_bone_dir = os.path.join('rsna-bone-age')
boneage_df = pd.read_csv(os.path.join(base_bone_dir, 'boneage-test-dataset.csv'))
boneage_df['path'] = boneage_df['fileName'].map(lambda x: os.path.join(base_bone_dir,
                                                         'boneage-test-dataset', 
                                                         'boneage-test-dataset', 
                                                         '{}'.format(x)))

In [8]:
boneage_df['exists'] = boneage_df['path'].map(os.path.exists)
print(boneage_df['exists'].sum(), 'images found of', boneage_df.shape[0], 'total')

1262 images found of 1262 total


In [ ]:
boneage_df[gender_str_col] = boneage_df[gender_str_col].map(
    lambda x: np.array([1]) if x else np.array([0])
    ) # map boolean values to 1 and 0

In [ ]:
# used a fixed dataset for evaluating the algorithm
gen_boneage = flow_from_dataframe(core_idg, valid_df_boneage, path_col='path',
                                        target_size=IMG_SIZE,
                                        color_mode='rgb',
                                        batch_size=BATCH_SIZE)

In [ ]:
def combined_generators(image_generator, gender_data, batch_size):
    gender_generator = cycle(batch(gender_data, batch_size))
    while True:
        nextImage = next(image_generator)
        nextGender = next(gender_generator)
        assert len(nextImage[0]) == len(nextGender)
        yield [nextImage[0], nextGender], nextImage[1]

In [ ]:
gen_wrapper = combined_generators(gen_boneage, boneage_df[gender_str_col], BATCH_SIZE_VAL)

In [ ]:
history = model.